# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [267]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [268]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [269]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [270]:
dfnew= data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': 'sum'})

In [271]:
dfnew

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [272]:
data2= data.pivot_table(index='ProductName',columns='CustomerID', values='Quantity', fill_value=0)

In [273]:
data2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [274]:
from scipy.spatial.distance import pdist, squareform
distances = pdist(data2.T, 'euclidean')
print(squareform(distances))

[[  0.          10.72380529   9.64365076 ... 197.94948851 218.8241303
  203.79892051]
 [ 10.72380529   0.          10.67707825 ... 197.34994299 218.62525014
  203.95342606]
 [  9.64365076  10.67707825   0.         ... 197.32460566 218.02981447
  203.80628057]
 ...
 [197.94948851 197.34994299 197.32460566 ...   0.         266.9269563
  266.9269563 ]
 [218.8241303  218.62525014 218.02981447 ... 266.9269563    0.
  269.25824036]
 [203.79892051 203.95342606 203.80628057 ... 266.9269563  269.25824036
    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [275]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))),
index=data2.columns, columns=data2.columns)


In [276]:

distances['CustomerID'] = pd.Series(200)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,CustomerID
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883,NaN
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879,NaN
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883,NaN
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886,NaN
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608,NaN
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765,NaN
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732,NaN


In [277]:
similarities = distances[33].sort_values(ascending=False)[1:6]
simil = list(similarities.index)

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [278]:

data3 = dfnew.loc[simil]

In [279]:
data3

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[282 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [280]:
data3= data3.groupby(['ProductName']).agg({'Quantity': 'sum'})

In [281]:
data4 = data3.sort_values(by='Quantity', ascending=False)

In [282]:
data4

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3
...,...
Gatorade - Xfactor Berry,1
Gloves - Goldtouch Disposable,1
Grouper - Fresh,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [283]:
data5 = dfnew.loc[[200]]
data5

Quantity
CustomerID ProductName                                
200        Bacardi Breezer - Tropical                1
           Bagel - Plain                             1
           Bar - Granola Trail Mix Fruit Nut         1
           Beef - Ground Medium                      1
           Beef - Montreal Smoked Brisket            1
...                                                ...
           Wiberg Super Cure                         1
           Wine - Gato Negro Cabernet                1
           Wine - Ruffino Chianti                    1
           Wine - White, Colubia Cresh               1
           Yogurt - Blueberry, 175 Gr                1

[72 rows x 1 columns]

In [284]:
data6 = data4.merge(data2[200],on=['ProductName'])

In [285]:
data7 = data6.loc[data6[200]== 0]

In [286]:
data7

,Quantity,200
ProductName,,
"Salsify, Organic",4,0
Chocolate - Dark,3,0
Wine - Charddonnay Errazuriz,3,0
Bay Leaf,3,0
"Oranges - Navel, 72",3,0
...,...,...
Gatorade - Xfactor Berry,1,0
Gloves - Goldtouch Disposable,1,0
Grouper - Fresh,1,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [287]:
allcust = {}
unique = list(data["CustomerID"].values)

In [288]:
unique

[61288,
 77352,
 40094,
 23548,
 78981,
 83106,
 11253,
 35107,
 15088,
 26031,
 26031,
 86595,
 46715,
 91013,
 69931,
 19753,
 9546,
 10224,
 72846,
 28976,
 3267,
 19260,
 76530,
 30196,
 48818,
 15166,
 11443,
 79060,
 82597,
 62411,
 42611,
 45420,
 83495,
 47122,
 30815,
 53198,
 18591,
 62034,
 1428,
 2187,
 55162,
 22345,
 81876,
 48641,
 81034,
 29681,
 87947,
 28867,
 69967,
 94420,
 83158,
 84364,
 44742,
 7772,
 47303,
 75791,
 77756,
 97324,
 97324,
 13912,
 52700,
 21734,
 6817,
 21095,
 28875,
 89588,
 1034,
 54827,
 29287,
 11102,
 52155,
 50748,
 88552,
 80938,
 60481,
 92168,
 66232,
 64348,
 37510,
 44408,
 89254,
 477,
 51483,
 97201,
 42157,
 52091,
 86065,
 18027,
 93882,
 42303,
 32010,
 53997,
 63033,
 76475,
 98185,
 84779,
 82146,
 54495,
 3903,
 3903,
 79854,
 25449,
 85496,
 95017,
 33759,
 43652,
 35873,
 97769,
 891,
 85161,
 25327,
 25624,
 68416,
 44090,
 71595,
 68336,
 38497,
 10016,
 36751,
 36751,
 61288,
 58421,
 45498,
 50774,
 79997,
 45313,
 5571

In [291]:
for c in unique:
    similarities = distances[c].sort_values(ascending=False)[1:6]
    simil = list(similarities.index)
    final = dfnew.loc[simil]
    data= final.groupby(['ProductName']).agg({'Quantity': 'sum'}).reset_index()
    data = data.sort_values(by='Quantity', ascending=False)
    data = data.merge(data2[c],on=['ProductName'])
    data = data.loc[data[c]== 0]
    data = data[1:6]
    data = list(data.ProductName)
    allcust[c] = data
    


KeyboardInterrupt: 

In [ ]:
allcust

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [292]:
dffinal = pd.DataFrame(allcust)

In [293]:
dffinal

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,48156,48655,51160,58203,51182,3531,44349,59379,16481,32028
0,"Wine - Magnotta, Merlot Sr Vqa",Grenadine,Pasta - Orecchiette,Banana Turning,Longos - Chicken Wings,Garlic - Peeled,"Juice - Cranberry, 341 Ml",Ice Cream Bar - Hageen Daz To,Blueberries,Beef - Top Sirloin,...,Assorted Desserts,Tilapia - Fillets,Eggplant - Asian,"Pepper - Paprika, Hungarian","Coconut - Shredded, Sweet",Ice Cream Bar - Hageen Daz To,Sea Bass - Whole,Pastry - Butterscotch Baked,Dried Figs,Garlic - Peeled
1,Chicken - Soup Base,Ecolab - Mikroklene 4/4 L,"Oregano - Dry, Rubbed",Towels - Paper / Kraft,Pop Shoppe Cream Soda,Soup - Campbells Tomato Ravioli,Tomatoes Tear Drop,Fuji Apples,Curry Paste - Madras,Cheese - Mozzarella,...,Pickerel - Fillets,Cheese - Camembert,"Beets - Candy Cane, Organic",Muffin - Zero Transfat,Flour - Whole Wheat,Table Cloth 54x72 White,Tray - 16in Rnd Blk,"Pork - Back, Short Cut, Boneless",Sausage - Breakfast,"Pork - Bacon, Double Smoked"
2,Wine - Chardonnay South,"Oranges - Navel, 72",Quiche Assorted,Bread - Raisin Walnut Oval,Sprouts - Alfalfa,Beef - Rib Eye Aaa,Lettuce - Treviso,Dc Hikiage Hira Huba,Soupfoamcont12oz 112con,Olive - Spread Tapenade,...,"Coconut - Shredded, Sweet",Wine - Valpolicella Masi,Ecolab - Lime - A - Way 4/4 L,Juice - V8 Splash,Zucchini - Yellow,Wine - Gato Negro Cabernet,Pickerel - Fillets,Beans - Wax,Curry Paste - Madras,Appetizer - Mushroom Tart
3,Milk - 1%,"Coconut - Shredded, Sweet",Beef - Inside Round,Lettuce - Treviso,Beef - Inside Round,Wine - Chablis 2003 Champs,Spinach - Baby,Beef - Prime Rib Aaa,Soup Knorr Chili With Beans,Garlic - Peeled,...,Cookie Dough - Double,Sea Bass - Whole,Cookie Dough - Double,Carbonated Water - Cherry,"Seedlings - Mix, Organic",Foam Dinner Plate,Milk Powder,Wasabi Powder,"Orange - Canned, Mandarin",Thermometer Digital
4,Bread - Multigrain,"Shrimp - Baby, Warm Water",Rabbit - Whole,Pernod,Wine - Gato Negro Cabernet,Bread Crumbs - Panko,Cumin - Whole,Wasabi Powder,Soup Campbells - Italian Wedding,Spinach - Baby,...,Wanton Wrap,Cornflakes,"Nut - Chestnuts, Whole",Puree - Mocha,Soup - Campbells Tomato Ravioli,Beer - Original Organic Lager,Grapes - Red,Kellogs Special K Cereal,Dc Hikiage Hira Huba,Beets - Mini Golden


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [294]:
from scipy.spatial.distance import pdist, squareform
distances = pdist(data2.T, 'jaccard')
print(squareform(distances))

[[0.         0.93495935 0.86915888 ... 1.         1.         1.        ]
 [0.93495935 0.         0.91935484 ... 1.         1.         1.        ]
 [0.86915888 0.91935484 0.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 0.         0.8976378  0.93442623]
 [1.         1.         1.         ... 0.8976378  0.         0.89230769]
 [1.         1.         1.         ... 0.93442623 0.89230769 0.        ]]


In [295]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'jaccard'))),
index=data2.columns, columns=data2.columns)

In [296]:
distances['CustomerID'] = pd.Series(200)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,CustomerID
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.516807,0.535000,0.526570,0.519824,0.520930,0.512712,0.513889,0.515837,0.518692,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,NaN
200,0.516807,1.000000,0.521008,0.518828,0.507463,0.512000,0.524590,0.510204,0.521008,0.523810,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,NaN
264,0.535000,0.521008,1.000000,0.518018,0.529412,0.515419,0.516949,0.508772,0.515419,0.513274,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,NaN
356,0.526570,0.518828,0.518018,1.000000,0.517094,0.513158,0.517094,0.524038,0.510823,0.513393,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,NaN
412,0.519824,0.507463,0.529412,0.517094,1.000000,0.516949,0.523013,0.525114,0.514644,0.522124,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.525581,0.515021,0.524510,0.532407,1.000000,0.532110,0.527778,0.515625,0.508065,NaN
98069,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.517241,0.516807,0.511013,0.531250,0.532110,1.000000,0.529412,0.521569,0.523404,NaN
98159,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.528302,0.522321,0.513889,0.521930,0.527778,0.529412,1.000000,0.526971,0.516949,NaN


In [297]:
allcust = {}
unique = list(data["CustomerID"].values)
for c in unique:
    similarities = distances[c].sort_values(ascending=False)[1:6]
    simil = list(similarities.index)
    final = dfnew.loc[simil]
    data= final.groupby(['ProductName']).agg({'Quantity': 'sum'}).reset_index()
    data = data.sort_values(by='Quantity', ascending=False)
    data = data.merge(data2[c],on=['ProductName'])
    data = data.loc[data[c]== 0]
    data = data[1:6]
    data = list(data.ProductName)
    allcust[c] = data
    


KeyError: 'CustomerID'

In [259]:
unique

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [255]:
for c in unique:
    similarities = distances[c].sort_values(ascending=False)[1:6]
    simil = list(similarities.index)
    final = dfnew.loc[simil]
    data= final.groupby(['ProductName']).agg({'Quantity': 'sum'}).reset_index()
    data = data.sort_values(by='Quantity', ascending=False)
    data = data.merge(data2[c],on=['ProductName'])
    data = data.loc[data[c]== 0]
    data = data[1:6]
    data = list(data.ProductName)
    allcust[c] = data
    

KeyError: nan

In [ ]:
dffinal = pd.DataFrame(allcust)